# Bootstrapping to find a 95% confidence interval

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datascience import *
%matplotlib inline

In [ ]:
faithful = Table().read_table("../data/faithful-new.csv")
faithful.show(3)

In [ ]:
faithful.num_rows

In [ ]:
faithful.hist('duration', bins=20)

In [ ]:
true_avg_duration = np.mean(faithful.column("duration"))
print(f"The true mean duration of Old Faithful geyser eruptions is {true_avg_duration:.2f} minutes")

## Old Faithful
### The true confidence interval for the mean based on a sample size of 50
This data set is clearly bimodal (two clusters), has 272 data points, and the mean duration of eruptions is 3.49 minutes. Let's pretend this is the true population and that 3.49 is the real population mean, but that we only have a sample data set of 50 data points. How well can we estimate the true mean from a sample of 50 points? 

Well, if we have the full data set, we can answer the true answer by sampling 50 data points over and over to find out confidence interval. This is analogous to performing an experiment over and over to get new data.

In [ ]:
our_data = faithful.sample(50, with_replacement=False)
our_mean = np.mean(our_data.column("duration"))

print("Based on a sample size of 50, we got a mean of:", our_mean)

In [ ]:
def one_sample_mean():
    """A function to sample the orginal data"""
    resample = faithful.sample(50, with_replacement=False)
    return np.mean(resample.column("duration"))

In [ ]:
num_repetitions = 5000
sample_means = make_array()
for i in np.arange(num_repetitions):
    sample_means = np.append(sample_means, one_sample_mean())

In [ ]:
plt.hist(sample_means, bins=20);

In [ ]:
# Get the endpoints of the 95% confidence interval
left = percentile(2.5, sample_means)
right = percentile(97.5, sample_means)

make_array(left, right)

So the true mean duration is 3.49 minutes, Our data, a sample of 50 values, yielded an estimate of this mean that was off, but with the 95% confidence interval.

This 95% confidence interval, however, we calculated using the full data set. In practice, we rarely have that, because we are unlikely to repeat our data collection thousands of times. What to do?

We bootstrap! We treat our sample of 50 data points as if it were the full dataset and resample it instead to estimate the confidence interval.

In [ ]:
def one_bootstrap_sample_mean():
    """A function to sample the orginal data"""
    resample = faithful.sample(with_replacement=True)
    return np.mean(resample.column("duration"))

In [ ]:
num_repetitions = 5000
bootstrap_sample_means = make_array()
for i in np.arange(num_repetitions):
    bootstrap_sample_means = np.append(bootstrap_sample_means, one_bootstrap_sample_mean())

In [ ]:
plt.hist(bootstrap_sample_means, bins=20);

In [ ]:
# Get the endpoints of the 95% confidence interval
left = percentile(2.5, bootstrap_sample_means)
right = percentile(97.5, bootstrap_sample_means)

make_array(left, right)

In [ ]:
# Our final result
print(f"The true mean of the Old Faithful eruption duration is {true_avg_duration:.2f}")
print("Based on a sample of 50 data points,")
print(f"the mean is {our_mean:.2f},")
print(f"with 95% confidence that the true mean lies between {left:.2f} and {right:.2f}")